Getting embeddings for categorical features

In [1]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
import os
import scipy.stats as ss
import scipy
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier, Pool, sum_models, to_classifier
from sklearn.model_selection import KFold

import implicit

In [2]:
#df2 = pd.read_csv("../input/irkutsk/train_3lags_semibalanced.csv", low_memory=False)

In [3]:
#df2 = pd.read_csv("../input/irkutsk/train_3lags_v4.csv", low_memory=False)

In [4]:
#df2.drop(['Unnamed: 0'], axis=1, inplace=True)

In [5]:
#df2 = df2.sample(frac=1).reset_index(drop=True)

In [6]:
X_clustered = pd.read_csv("../input/irkutsk/df_train_clustered.csv", low_memory=False)

In [7]:
df_test = X_clustered[['service_title']]

In [8]:
#df_test = df2[['service_title', 'order_count']]

In [9]:
df_train = X_clustered.drop(['index', 'age', 'cluster', 'service_title'], axis=1)

In [10]:
df_train.reset_index(inplace=True)
df_test.reset_index(inplace=True)

In [11]:
df_train.drop(['index'], axis=1, inplace=True)
df_test.drop(['index'], axis=1, inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
#mask = (df_train_mask['order_count'] == 5)
#z_valid = df_train_mask[mask]

#df_train_mask.loc[mask, 'service_4'] = np.nan


In [13]:
df_test

,service_title
0,907
1,1259
2,263
3,107
4,234
...,...
344887,1259
344888,604
344889,603
344890,603


In [14]:
df_train.columns

Index(['service_1', 'service_title_1', 'mfc_1', 'internal_status_1',
       'external_status_1', 'order_type_1', 'department_id_1',
       'custom_service_id_1', 'service_level_1', 'is_subdep_1', 'is_csid_1',
       'dayofweek_1', 'day_part_1', 'month_1', 'week_1', 'year_1', 'person_1',
       'sole_1', 'legal_1', 'auto_ping_queue_1', 'requester_type', 'gender'],
      dtype='object')

In [15]:
categorical = ['service_1', 'service_title_1', 'mfc_1',
       'internal_status_1', 'external_status_1', 'order_type_1',
       'department_id_1', 'custom_service_id_1', 'service_level_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1', 'day_part_1', 'month_1', 'week_1', 'year_1',
       'person_1', 'sole_1', 'legal_1', 'auto_ping_queue_1',
       'service_2', 'service_title_2', 'mfc_2', 'internal_status_2',
       'external_status_2', 'order_type_2', 'department_id_2',
       'custom_service_id_2', 'service_level_2', 'is_subdep_2', 'is_csid_2',
       'dayofweek_2', 'day_part_2', 'person_2', 'sole_2', 'month_2', 'week_2', 'year_2',
       'legal_2', 'auto_ping_queue_2','service_3',
       'service_title_3', 'mfc_3', 'internal_status_3', 'external_status_3',
       'order_type_3', 'department_id_3', 'custom_service_id_3',
       'service_level_3', 'is_subdep_3', 'is_csid_3', 'month_3', 'week_3', 'year_3',
       'dayofweek_3', 'day_part_3', 'person_3', 'sole_3', 'legal_3',
       'auto_ping_queue_3',
       'requester_type', 'gender']

In [16]:
cat = ['service_1', 'service_title_1', 'mfc_1',
       'internal_status_1', 'external_status_1',
       'department_id_1', 'custom_service_id_1', 'month_1', 'week_1', 'year_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1',
       'service_2', 'service_title_2', 'mfc_2', 'internal_status_2',
       'external_status_2', 'department_id_2', 'month_2', 'week_2', 'year_2',
       'custom_service_id_2', 'is_subdep_2', 'is_csid_2',
       'dayofweek_2', 'service_3',
       'service_title_3', 'mfc_3', 'internal_status_3', 'external_status_3',
       'department_id_3', 'custom_service_id_3',
       'is_subdep_3', 'is_csid_3', 'month_3', 'week_3', 'year_3',
       'dayofweek_3', 
       'requester_type', 'gender']

In [17]:
X = df_train
y = df_test['service_title']

In [18]:
categorical = ['service_1', 'service_title_1', 'mfc_1',
       'internal_status_1', 'external_status_1', 'order_type_1',
       'department_id_1', 'custom_service_id_1', 'service_level_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1', 'day_part_1', 'month_1', 'week_1', 'year_1',
       'person_1', 'sole_1', 'legal_1', 'auto_ping_queue_1',
       'requester_type', 'gender']

cat = ['service_1', 'service_title_1', 'mfc_1',
       'internal_status_1', 'external_status_1',
       'department_id_1', 'custom_service_id_1', 'month_1', 'week_1', 'year_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1', 
       'requester_type', 'gender']

In [19]:
X[cat] = X[cat].astype('Int64')
X[cat] = X[cat].astype('object')

In [20]:
#X[cat] = X[cat].astype('Int64')
#X[cat] = X[cat].astype('object')

In [21]:
X[categorical] = X[categorical].fillna('NA')

In [22]:
def reduce_mem_usage(df):
    """ 
    iterate through all the columns of a dataframe and 
    modify the data type to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage of dataframe is {:.2f}' 
                     'MB').format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max <\
                  np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max <\
                   np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max <\
                   np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max <\
                   np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max <\
                   np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max <\
                   np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage after optimization is: {:.2f}' 
                              'MB').format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) 
                                             / start_mem))
    
    return df

In [23]:
X = reduce_mem_usage(X)

Memory usage of dataframe is 57.89MB
Memory usage after optimization is: 13.82MB
Decreased by 76.1%


In [24]:
X = X[categorical]

In [25]:
X['person_1'] = X['person_1'].astype('int32')
X['sole_1'] = X['sole_1'].astype('int32')
X['legal_1'] = X['legal_1'].astype('int32')
X['auto_ping_queue_1'] = X['auto_ping_queue_1'].astype('int32')

In [26]:
from sklearn import preprocessing
labeling = []
for col in X.columns:
    d = pd.DataFrame()
    d[col] = X[col].unique()
    le = preprocessing.LabelEncoder()
    le.fit(d[col])
    d[col+'_l'] = le.transform(d[col])
    d = d.sort_values(by=[col+'_l']).reset_index(drop=True)
    labeling.append(d)

In [27]:
d = pd.DataFrame()
d['service_title'] = y.unique()
le = preprocessing.LabelEncoder()
le.fit(d['service_title'])
d['service_title'+'_l'] = le.transform(d['service_title'])
d = d.sort_values(by=['service_title'+'_l']).reset_index(drop=True)
labeling_y = d

In [28]:
labeling[0]

,service_1,service_1_l
0,43,0
1,2266851,1
2,2266858,2
3,2266865,3
4,2266872,4
5,46880280,5
6,64279142,6
7,64279302,7
8,64279389,8
9,64279525,9


In [29]:
i = 0
for col in X.columns:
    X[col] = X[col].map(labeling[i].set_index(col).to_dict()[col+'_l'])
    i += 1

In [30]:
X

,service_1,service_title_1,mfc_1,internal_status_1,external_status_1,order_type_1,department_id_1,custom_service_id_1,service_level_1,is_subdep_1,...,day_part_1,month_1,week_1,year_1,person_1,sole_1,legal_1,auto_ping_queue_1,requester_type,gender
0,4,757,127,7,3,3,15,0,0,0,...,1,0,4,1,1,0,0,0,0,1
1,13,541,1,7,17,3,63,0,0,0,...,1,3,15,1,1,0,0,1,0,0
2,0,156,2,7,17,3,68,33,3,0,...,1,3,15,0,0,0,0,0,0,0
3,0,12,62,7,17,3,16,8,4,0,...,1,7,31,0,0,0,0,1,0,1
4,0,73,201,7,17,3,8,28,0,0,...,0,1,4,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344887,3,545,4,7,3,3,15,0,0,0,...,1,1,6,0,1,0,0,1,0,0
344888,0,239,138,7,17,3,26,1,1,0,...,2,5,24,0,0,0,0,0,0,1
344889,0,358,140,7,17,3,63,25,0,0,...,1,4,21,0,0,0,0,1,0,1
344890,0,137,133,7,17,3,63,17,0,0,...,0,1,6,0,0,0,0,1,0,1


In [31]:
y = y.map(labeling_y.set_index('service_title').to_dict()['service_title'+'_l'])

In [32]:
for col in X.columns:
    print(col, len(X[col].unique()))

service_1 17
service_title_1 785
mfc_1 212
internal_status_1 9
external_status_1 23
order_type_1 4
department_id_1 78
custom_service_id_1 51
service_level_1 5
is_subdep_1 2
is_csid_1 2
dayofweek_1 7
day_part_1 4
month_1 12
week_1 52
year_1 2
person_1 2
sole_1 2
legal_1 2
auto_ping_queue_1 2
requester_type 3
gender 2


In [33]:
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold

In [34]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [35]:
AUTO = tf.data.experimental.AUTOTUNE

BATCH_SIZE = 32 * strategy.num_replicas_in_sync
LEARNING_RATE = 1e-3 * strategy.num_replicas_in_sync 
EPOCHS = 5

In [36]:
def build_model():
    service_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='service_1')
    service_title_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='service_title_1')
    mfc_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='mfc_1')
    internal_status_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='internal_status_1')
    external_status_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='external_status_1')
    order_type_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='order_type_1')
    department_id_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='department_id_1')
    custom_service_id_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='custom_service_id_1')
    service_level_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='service_level_1')
    is_subdep_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='is_subdep_1')
    is_csid_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='is_csid_1')
    dayofweek_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='dayofweek_1')
    day_part_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='day_part_1')
    month_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='month_1')
    week_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='week_1')
    year_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='year_1')
    person_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='person_1')
    sole_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='sole_1')
    legal_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='legal_1')
    auto_ping_queue_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='auto_ping_queue_1')
    requester_type = tf.keras.layers.Input((1,), dtype=tf.int32, name='requester_type')
    gender = tf.keras.layers.Input((1,), dtype=tf.int32, name='gender')
    
    service_1_embedding = tf.keras.layers.Embedding(len(X['service_1'].unique()), 11, input_length=1, name='service_1_embedding')(service_1)
    service_title_1_embedding = tf.keras.layers.Embedding(len(X['service_title_1'].unique()), 50, input_length=1, name='service_title_1_embedding')(service_title_1)
    mfc_1_embedding = tf.keras.layers.Embedding(len(X['mfc_1'].unique()), 50, input_length=1, name='mfc_1_embedding')(mfc_1)
    internal_status_1_embedding = tf.keras.layers.Embedding(len(X['internal_status_1'].unique()), 5, input_length=1, name='internal_status_1_embedding')(internal_status_1)
    external_status_1_embedding = tf.keras.layers.Embedding(len(X['external_status_1'].unique()), 14, input_length=1, name='external_status_1_embedding')(external_status_1)
    order_type_1_embedding = tf.keras.layers.Embedding(len(X['order_type_1'].unique()), 2, input_length=1, name='order_type_1_embedding')(order_type_1)
    department_id_1_embedding = tf.keras.layers.Embedding(len(X['department_id_1'].unique()), 46, input_length=1, name='department_id_1_embedding')(department_id_1)
    custom_service_id_1_embedding = tf.keras.layers.Embedding(len(X['custom_service_id_1'].unique()), 26, input_length=1, name='custom_service_id_1_embedding')(custom_service_id_1)
    service_level_1_embedding = tf.keras.layers.Embedding(len(X['service_level_1'].unique()), 3, input_length=1, name='service_level_1_embedding')(service_level_1)
    is_subdep_1_embedding = tf.keras.layers.Embedding(len(X['is_subdep_1'].unique()), 1, input_length=1, name='is_subdep_1_embedding')(is_subdep_1)
    is_csid_1_embedding = tf.keras.layers.Embedding(len(X['is_csid_1'].unique()), 1, input_length=1, name='is_csid_1_embedding')(is_csid_1)
    dayofweek_1_embedding = tf.keras.layers.Embedding(len(X['dayofweek_1'].unique()), 4, input_length=1, name='dayofweek_1_embedding')(dayofweek_1)
    day_part_1_embedding = tf.keras.layers.Embedding(len(X['day_part_1'].unique()), 2, input_length=1, name='day_part_1_embedding')(day_part_1)
    month_1_embedding = tf.keras.layers.Embedding(len(X['month_1'].unique()), 6, input_length=1, name='month_1_embedding')(month_1)
    week_1_embedding = tf.keras.layers.Embedding(len(X['week_1'].unique()), 26, input_length=1, name='week_1_embedding')(week_1)
    year_1_embedding = tf.keras.layers.Embedding(len(X['year_1'].unique()), 1, input_length=1, name='year_1_embedding')(year_1)
    person_1_embedding = tf.keras.layers.Embedding(len(X['person_1'].unique()), 1, input_length=1, name='person_1_embedding')(person_1)
    sole_1_embedding = tf.keras.layers.Embedding(len(X['sole_1'].unique()), 1, input_length=1, name='sole_1_embedding')(sole_1)
    legal_1_embedding = tf.keras.layers.Embedding(len(X['legal_1'].unique()), 1, input_length=1, name='legal_1_embedding')(legal_1)
    auto_ping_queue_1_embedding = tf.keras.layers.Embedding(len(X['auto_ping_queue_1'].unique()), 1, input_length=1, name='auto_ping_queue_1_embedding')(auto_ping_queue_1)
    requester_type_embedding = tf.keras.layers.Embedding(len(X['requester_type'].unique()), 2, input_length=1, name='requester_type_embedding')(requester_type)
    gender_embedding = tf.keras.layers.Embedding(len(X['gender'].unique()), 1, input_length=1, name='gender_embedding')(gender)

    concatenated = tf.keras.layers.Concatenate()([service_1_embedding, 
                                                  service_title_1_embedding,
                                                 mfc_1_embedding,
                                                 internal_status_1_embedding,
                                                 external_status_1_embedding,
                                                 order_type_1_embedding,
                                                 department_id_1_embedding,
                                                 custom_service_id_1_embedding,
                                                 service_level_1_embedding,
                                                 is_subdep_1_embedding,
                                                 is_csid_1_embedding,
                                                 dayofweek_1_embedding,
                                                 day_part_1_embedding,
                                                 month_1_embedding,
                                                 week_1_embedding,
                                                 year_1_embedding,
                                                 person_1_embedding,
                                                 sole_1_embedding,
                                                 legal_1_embedding,
                                                 auto_ping_queue_1_embedding,
                                                 requester_type_embedding,
                                                 gender_embedding])
    #out = tf.keras.layers.Flatten()(concatenated)
    
    #out = tf.keras.layers.Dense(512, activation='relu')(out)
    #out = tf.keras.layers.Dense(256, activation='relu')(out)
    #out = tf.keras.layers.Dense(256, activation='relu')(out)
    #out = tf.keras.layers.Conv1D(128, 2, padding='same')(concatenated)
    #out = tf.keras.layers.LeakyReLU()(out)
    #out = tf.keras.layers.Conv1D(64, 2, padding='same')(out)
    #out = tf.keras.layers.Flatten()(out)
    conv_0 = tf.keras.layers.Conv1D(128, 3, padding='same', activation='relu')(concatenated)
    conv_1 = tf.keras.layers.Conv1D(128, 2, padding='same', activation='relu')(concatenated)
    conv_2 = tf.keras.layers.Conv1D(128, 6, padding='same', activation='relu')(concatenated)
    conv_0 = tf.keras.layers.Conv1D(64, 3, padding='same', activation='relu')(conv_0)
    conv_1 = tf.keras.layers.Conv1D(64, 2, padding='same', activation='relu')(conv_1)
    conv_2 = tf.keras.layers.Conv1D(64, 6, padding='same', activation='relu')(conv_2)
    concatenated_tensor = tf.keras.layers.Concatenate(axis=1)([conv_0, conv_1, conv_2])
    out = tf.keras.layers.Flatten()(concatenated_tensor)
    
    out = tf.keras.layers.Dropout(0.1)(out)
    
    out = tf.keras.layers.Dense(len(y.unique()), activation='softmax', name='service_title')(out)

    model = tf.keras.models.Model(inputs=[service_1, 
                                          service_title_1, 
                                          mfc_1,
                                         internal_status_1,
                                         external_status_1,
                                         order_type_1,
                                         department_id_1,
                                         custom_service_id_1,
                                         service_level_1,
                                         is_subdep_1,
                                         is_csid_1,
                                         dayofweek_1,
                                         day_part_1,
                                         month_1,
                                         week_1,
                                         year_1,
                                         person_1,
                                         sole_1,
                                         legal_1,
                                         auto_ping_queue_1,
                                         requester_type,
                                         gender], 
                                  outputs=out)

    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(loss = 'categorical_crossentropy', optimizer=optimizer, metrics=["accuracy"])

    return model

In [37]:
model = build_model()
model.load_weights('../input/emb-weights/v9-learnedemb-0.h5')

In [38]:
X.columns

Index(['service_1', 'service_title_1', 'mfc_1', 'internal_status_1',
       'external_status_1', 'order_type_1', 'department_id_1',
       'custom_service_id_1', 'service_level_1', 'is_subdep_1', 'is_csid_1',
       'dayofweek_1', 'day_part_1', 'month_1', 'week_1', 'year_1', 'person_1',
       'sole_1', 'legal_1', 'auto_ping_queue_1', 'requester_type', 'gender'],
      dtype='object')

In [39]:
embeddings = []
j = 0
for col in X.columns:
    weights = model.get_layer(col + '_embedding').get_weights()[0].transpose()
    d = labeling[j][:]
    for i in range(weights.shape[0]):
           d[col + '_e_' + str(i)] = weights[i]
    j += 1
    embeddings.append(d)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [42]:
X_emb = X_clustered[:]
i = 0
for col in X.columns:
    X_emb = pd.merge(X_emb, embeddings[i], how='left', on=col)
    i += 1

In [43]:
X_emb.drop(['index', 'service_1', 'service_title_1', 'mfc_1', 'internal_status_1',
       'external_status_1', 'order_type_1', 'department_id_1',
       'custom_service_id_1', 'service_level_1', 'is_subdep_1', 'is_csid_1',
       'dayofweek_1', 'day_part_1', 'month_1', 'week_1', 'year_1', 'person_1',
       'sole_1', 'legal_1', 'auto_ping_queue_1', 'requester_type', 'gender', 'cluster'], axis=1, inplace=True)

In [44]:
X_emb.drop(['service_1_l', 'service_title_1_l', 'mfc_1_l', 'internal_status_1_l',
       'external_status_1_l', 'order_type_1_l', 'department_id_1_l',
       'custom_service_id_1_l', 'service_level_1_l', 'is_subdep_1_l', 'is_csid_1_l',
       'dayofweek_1_l', 'day_part_1_l', 'month_1_l', 'week_1_l', 'year_1_l', 'person_1_l',
       'sole_1_l', 'legal_1_l', 'auto_ping_queue_1_l', 'requester_type_l', 'gender_l'], axis=1, inplace=True)

In [46]:
X_emb.to_csv('df_train_emb.csv', index=False)